In [2]:
from pathlib import Path
import matplotlib.pyplot as plt

parent_directory = str(Path().resolve().parent)
parent_directory

import os,sys 
script_path = os.path.abspath("../src")  # Adjust the path
sys.path.append(script_path)
print(parent_directory)
import doc_provenance

/Users/yiminglin/Documents/Codebase/doc-provenance


[nltk_data] Downloading package punkt to /Users/yiminglin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
import json 
import tiktoken

def read_json(path):
    if not os.path.exists(path):
        return None
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    return data

def count_tokens(text, model="gpt-4o-mini"):
    encoder = tiktoken.encoding_for_model(model)  # Get the tokenizer for the specific model
    tokens = encoder.encode(text)  # Encode text into tokens
    return len(tokens)

In [ ]:
#paper dataset 
data_path = parent_directory + '/out/papers/results/'
strategies = ['vallina_LLM','sequential_greedy','divide_and_conquer','heuristic_greedy','heuristic_topk','exponential_greedy','divide_and_conquer_progressive']
strategy = 'divide_and_conquer_progressive'

doc_num = 4
q_num = 3
runs = {}
for d_id in range(doc_num):
    for q_id in range(q_num):
        #this is one run 
        file_path = data_path + 'doc' + str(d_id) + '_q' + str(q_id) + '_' + strategy + '.json'
        result = read_json(file_path)
        if(result == None):
            continue
        if strategy not in runs:
            runs[strategy] = []
        runs[strategy].append(result) 



[{'title': '"I Don\'t Understand..." Issues in Self-Quantifying Commuting', 'question': ['In what year was this paper published?', 'Return only a number. Do not add explanations.'], 'answers': ['2019'], 'path': '/Users/yiminglin/Documents/Codebase/doc-provenance/out/papers/results/doc0_q0_divide_and_conquer_progressive.json', 'context_size': 5912, 'time': 18.206860065460205, 'time_breakdown': {'0': 3.425351142883301, '1': 5.61928915977478, '2': 9.694602966308594, '3': 14.777266263961792, '4': 18.206812143325806}, 'tokens': [26563, 113], 'tokens_breakdown': {'0': [12887, 16], '1': [13871, 24], '2': [15737, 72], '3': [25395, 92], '4': [26563, 113]}, 'provenance_ids_breakdown': {'0': [6], '1': [15], '2': [32], '3': [113], '4': [93]}}, {'title': '"I Don\'t Understand..." Issues in Self-Quantifying Commuting', 'question': ['Who are the authors of this paper?', 'Return only the author names. Do not add explanations.'], 'answers': ['Cecile Boulard, Stefania Castellani, Tommaso Colombino, Anto

In [18]:
c = 0
input_cost_unit = 0.15/1000000
output_cost_unit = 0.6/1000000

sum_top1_time = 0
sum_total_time = 0
sum_top1_cost = 0
sum_total_cost = 0
sum_k = 0

for run in runs[strategy]:
    time_breakdown = run['time_breakdown']
    tokens_breakdown = run['tokens_breakdown']
    k = len(tokens_breakdown)
    if k == 0:
        continue
    # print(tokens_breakdown)
    # print(time_breakdown)
    if '0' not in time_breakdown:
        continue
    top1_time = time_breakdown['0']
    top1_cost = input_cost_unit*tokens_breakdown['0'][0] + output_cost_unit*tokens_breakdown['0'][1]
    total_time = time_breakdown[str(k-1)]
    total_cost = input_cost_unit*tokens_breakdown[str(k-1)][0] + output_cost_unit*tokens_breakdown[str(k-1)][1]
    
    if top1_time > 10:
        continue
    print(k, 'top1_time:',top1_time,'total_time:',total_time,'top1_cost:',top1_cost,'total_cost:',total_cost)
    sum_top1_time += top1_time
    sum_total_time += total_time
    sum_top1_cost += top1_cost
    sum_total_cost += total_cost
    sum_k += k
    c+=1
    #break
avg_top1_time = sum_top1_time/c
avg_total_time = sum_total_time/c
avg_top1_cost = sum_top1_cost/c
avg_total_cost = sum_total_cost/c
avg_k = sum_k/c

print(avg_k, avg_top1_time,avg_total_time,avg_top1_cost,avg_total_cost)

5 top1_time: 3.425351142883301 total_time: 18.206812143325806 top1_cost: 0.00194265 total_cost: 0.004052249999999999
1 top1_time: 7.5308287143707275 total_time: 7.5308287143707275 top1_cost: 0.00196605 total_cost: 0.00196605
5 top1_time: 4.256409168243408 total_time: 18.610926151275635 top1_cost: 0.005650349999999999 total_cost: 0.007624199999999999
1 top1_time: 9.515817880630493 total_time: 9.515817880630493 top1_cost: 0.00578535 total_cost: 0.00578535
5 top1_time: 8.485649824142456 total_time: 31.22486400604248 top1_cost: 0.00398325 total_cost: 0.0053776499999999994
5 top1_time: 8.511884927749634 total_time: 38.40840196609497 top1_cost: 0.00406995 total_cost: 0.007076849999999999
5 top1_time: 6.511491060256958 total_time: 29.61546802520752 top1_cost: 0.0037259999999999997 total_cost: 0.00659925
5 top1_time: 7.789639949798584 total_time: 38.068262815475464 top1_cost: 0.0039735 total_cost: 0.0084441
4.0 7.003384083509445 23.897672712802887 0.0038871374999999994 0.005865712499999998


In [19]:
#hotpot dataset 
data_path = parent_directory + '/out/hotpotQA/results/'
strategies = ['vallina_LLM','sequential_greedy','divide_and_conquer','heuristic_greedy','heuristic_topk','exponential_greedy','divide_and_conquer_progressive']
strategy = 'divide_and_conquer_progressive'


q_num = 12
runs = {}
for q_id in range(q_num):
    #this is one run 
    file_path = data_path + 'hotpot' + '_q' + str(q_id) + '_' + strategy + '.json'
    result = read_json(file_path)
    if strategy not in runs:
        runs[strategy] = []
    runs[strategy].append(result) 

In [20]:
c = 0
input_cost_unit = 0.15/1000000
output_cost_unit = 0.6/1000000

sum_top1_time = 0
sum_total_time = 0
sum_top1_cost = 0
sum_total_cost = 0
sum_k = 0

for run in runs[strategy]:
    time_breakdown = run['time_breakdown']
    tokens_breakdown = run['tokens_breakdown']
    k = len(tokens_breakdown)
    if k == 0:
        continue
    # print(tokens_breakdown)
    # print(time_breakdown)
    if '0' not in time_breakdown:
        continue
    top1_time = time_breakdown['0']
    top1_cost = input_cost_unit*tokens_breakdown['0'][0] + output_cost_unit*tokens_breakdown['0'][1]
    total_time = time_breakdown[str(k-1)]
    total_cost = input_cost_unit*tokens_breakdown[str(k-1)][0] + output_cost_unit*tokens_breakdown[str(k-1)][1]
    
    if top1_time > 10:
        continue
    print(k, 'top1_time:',top1_time,'total_time:',total_time,'top1_cost:',top1_cost,'total_cost:',total_cost)
    sum_top1_time += top1_time
    sum_total_time += total_time
    sum_top1_cost += top1_cost
    sum_total_cost += total_cost
    sum_k += k
    c+=1
    #break
avg_top1_time = sum_top1_time/c
avg_total_time = sum_total_time/c
avg_top1_cost = sum_top1_cost/c
avg_total_cost = sum_total_cost/c
avg_k = sum_k/c

print(avg_k, avg_top1_time,avg_total_time,avg_top1_cost,avg_total_cost)

4 top1_time: 1.9364728927612305 total_time: 6.123777866363525 top1_cost: 0.00025739999999999997 total_cost: 0.00041699999999999994
1 top1_time: 6.022511959075928 total_time: 6.022511959075928 top1_cost: 0.00039855 total_cost: 0.00039855
1 top1_time: 2.9472336769104004 total_time: 2.9472336769104004 top1_cost: 0.0005778000000000001 total_cost: 0.0005778000000000001
5 top1_time: 2.791282892227173 total_time: 11.525734186172485 top1_cost: 0.0008013 total_cost: 0.00122445
4 top1_time: 4.159999847412109 total_time: 10.54534387588501 top1_cost: 0.00046619999999999995 total_cost: 0.0006266999999999999
5 top1_time: 4.366109848022461 total_time: 16.207197189331055 top1_cost: 0.00045929999999999994 total_cost: 0.0006603
5 top1_time: 6.016167879104614 total_time: 24.559802770614624 top1_cost: 0.0004881 total_cost: 0.0010134
1 top1_time: 4.546765089035034 total_time: 4.546765089035034 top1_cost: 0.00033224999999999997 total_cost: 0.00033224999999999997
3.25 4.098318010568619 10.309795826673508 0.0